In [ ]:
import os
import cv2
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def pointLoad(imgpath,pts):
    if os.path.exists(imgpath):
        img = cv2.imread(imgpath)
        plt.imshow(img)
        ptMatrix = plt.ginput(pts)
        # plt.close()
        return img, np.array(ptMatrix).astype(int)
        

    else:
        print("Invalid image path")
        return

In [ ]:
def getH(dest, src):
    p = np.array([0,0,0,0,0,0,0,0,1])
    # if len(dest) < 5 and len(src) < 5:
        

    for i in range(len(dest)):
        pi = np.array([[-src[i][0],-src[i][1],-1,0,0,0,src[i][0]*dest[i][0],src[i][1]*dest[i][0],dest[i][0]],
        [0,0,0,-src[i][0],-src[i][1],-1,src[i][0]*dest[i][1],src[i][1]*dest[i][1],dest[i][1]]])
        p = np.vstack((pi,p))

  
    x = np.zeros((len(dest)*2,1))
    x = np.vstack((x,np.array([1])))
    H, residuals, rank, s = np.linalg.lstsq(p,x)
    H = H.reshape((3,3))
    

    
    return H
        


In [27]:
def pano(image_paths):
    imgs = []
    imgs_gray = []
    orbvals={"keypts":[],"descrpt":[]}
    orb = cv2.ORB_create()
    # sift = cv2.xfeatures2d.SIFT_create(nfeatures=200)
    for i in range(len(image_paths)):
        imgs.append(cv2.imread(image_paths[i]))
        imgs[i]=cv2.resize(imgs[i],(0,0),fx=0.4,fy=0.4)
        imgs_gray.append(cv2.cvtColor(imgs[i],cv2.COLOR_BGR2GRAY))
        keypt, desc = orb.detectAndCompute(imgs_gray[i],None)
        # keypt, desc = sift.detectAndCompute(imgs_gray[i], None)
        orbvals["keypts"].append(keypt)
        orbvals["descrpt"].append(desc)
        print((orbvals["keypts"]))
        cv2.imshow("image",cv2.drawKeypoints(imgs[i], orbvals["keypts"][i], None, (255, 0, 255)))





In [ ]:
img1, img1pts = pointLoad("image1.jpg",8)

img2, img2pts = pointLoad("image2.jpg",8)




In [ ]:

cv2.imshow("image 1",img1)
cv2.imshow("image 2",img2)


In [ ]:
H = getH(img1pts,img2pts)
im_out = cv2.warpPerspective(img2,H,(img1.shape[1],img1.shape[0]))
mask = np.zeros(img1.shape[:2], dtype="uint8")
cv2.fillConvexPoly(mask, np.int32([img1pts]),color=255)
cv2.fillConvexPoly(img1, np.int32([img1pts]),color=0)
masked =cv2.bitwise_and(im_out,im_out,mask=mask)
img3 = cv2.add(img1,masked)
plt.imshow(img3)

plt.show()

In [28]:

# image_paths=['s1.jpeg','s2.jpeg','s3.jpeg','s4.jpeg','s5.jpeg']
image_paths=['s1.jpeg','s2.jpeg']


# stitchy=cv2.Stitcher.create()
# (dummy,output)=stitchy.stitch(imgs)

# if dummy != cv2.STITCHER_OK:

# 	print("stitching ain't successful")
# else:
# 	print('Your Panorama is ready!!!')

pano(image_paths)


# final output
# cv2.imshow('final result',output)

# cv2.waitKey(0)


[(<KeyPoint 0x19f7681b0>, <KeyPoint 0x19f768420>, <KeyPoint 0x19f768630>, <KeyPoint 0x19f768390>, <KeyPoint 0x19f768180>, <KeyPoint 0x19f768330>, <KeyPoint 0x18d6f0720>, <KeyPoint 0x19ecdfe40>, <KeyPoint 0x19ecdff90>, <KeyPoint 0x19eca5480>, <KeyPoint 0x19eca57e0>, <KeyPoint 0x19eca5900>, <KeyPoint 0x19eca50f0>, <KeyPoint 0x19eca55d0>, <KeyPoint 0x19ed6ab70>, <KeyPoint 0x19ed6aed0>, <KeyPoint 0x19ed6af90>, <KeyPoint 0x19ed6ac90>, <KeyPoint 0x19ed6a0f0>, <KeyPoint 0x19ed6a810>, <KeyPoint 0x19ed6aab0>, <KeyPoint 0x19ed6a240>, <KeyPoint 0x19ec9abd0>, <KeyPoint 0x19ec9aa20>, <KeyPoint 0x19ec9a840>, <KeyPoint 0x19ec9aab0>, <KeyPoint 0x19ec9a900>, <KeyPoint 0x19ec9ab10>, <KeyPoint 0x19ec9ab40>, <KeyPoint 0x19f872030>, <KeyPoint 0x19f872060>, <KeyPoint 0x19f872090>, <KeyPoint 0x19f8720c0>, <KeyPoint 0x19f8720f0>, <KeyPoint 0x19f872120>, <KeyPoint 0x19f872150>, <KeyPoint 0x19f872180>, <KeyPoint 0x19f8721b0>, <KeyPoint 0x19f8721e0>, <KeyPoint 0x19f872210>, <KeyPoint 0x19f872240>, <KeyPoint 0x19